In [1]:
Company.objects.all()

<QuerySet [<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>, <Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>]>

In [2]:
!cat .env

STAGE=CCR_PRODUCTION


In [3]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
import os
from PIL import Image
from io import BytesIO
import datetime
import json
import re
import pandas
import openpyxl
from uuid import uuid4
from helpers.signals import DisableSignals
from helpers.apps.reportings import form_fields_dict
from helpers.strings import get_obj_from_path
from helpers.histories import bulk_update_with_history
from simple_history.utils import bulk_create_with_history

In [4]:
def load_excel(filename,sheetPosision=0):
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[wb.sheetnames[sheetPosision]]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        img={}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value
        data.append(row_dict)
    return data,sheet

In [5]:
def ret_campo_foto_array(classe):
    campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[a['apiName']] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[a['apiName']].append({"inner_api":inner['apiName'],
                                               "main_array":a['displayName'],"inner_display":inner['displayName']})
            campos.append(temp)
    return campos

In [6]:
def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [7]:
def get_photo_kind(kind):
    company=Company.objects.get(name="CCR - Rio Sp")
    valor = next(a['value'] for a in company.custom_options['reportingFile']['fields']['kind']['selectOptions']['options'] if a['name'] == kind)
    return valor

In [8]:
def convert_images_to_jpg(root_dir):
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            filepath = os.path.join(subdir, file)
            if filepath.endswith(".png") or filepath.endswith(".jpeg"):
                with Image.open(filepath) as im:
                    im.convert("RGB").save(filepath.replace(".png", ".jpg").replace(".jpeg", ".jpg"))
                os.remove(filepath)

In [9]:
filename='CÓPIAMonitoração de Terraplenos 2023'
planilha,_=load_excel(filename, 1)

In [10]:
# exemplo: 'CCR_Monitoracao/marcas viarias/RJ/Sul/'
# s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/2024/eps_sin_horizontal/Dispositivos/
# s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/dreProf/
caminho_imagens='arquivos_monitoracao/2024/ocupacoes_v4/'
# convert_images_to_jpg(caminho_imagens)s3://kartado-ccr-production-assets/media/private/arquivos_monitoracao/2024/ocupacoes/

# Se o caminho utiliza identificação antt e a quantidades maxima de fotos

In [11]:
novo_caminho=False
qtd_fotos_por_linhas = 2

# Somente Fotos Upload

In [12]:
user=User.objects.get(username="rlcs")
company=Company.objects.get(name='CCR - Rio Sp')
# _,sheet=load_excel('Template - Monitoração - Barreira - BR-116', 1)

In [13]:
len(planilha),planilha[0]

(2099,
 {'Código Apontamento': 'CCR-TER-2024.00001',
  'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.30920',
  'Foto de Croqui_1: Adicionar Foto de Croqui - Foto_1': 'TP 101 RJ 380+800 N1.WMF'})

In [14]:
ret_campo_foto_array(OccurrenceType.objects.get(name=planilha[0]['Classe']))

KeyError: 'Classe'

# Rodar pra erros da planilha por multiples valores retornados no get
# Feito pelo Serial do apontamento

In [15]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha)):
    try:
        ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
#         ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Código da Agência')
        try:
#             parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
            number=a['Código Apontamento'] if a.get('Código Apontamento') else a.get('Código Apontamento ')
            rep=Reporting.objects.get(
#                                         parent__number=parent.strip(),
#                                         form_data__id_ccr_antt=ficha.strip(),
#                                         firm__name=a['Equipe'].strip(),
                                        number=number,
#                                       km=a['km'],
        #                               occurrence_type__name=a['Classe'].strip(),
                                        company=company)
        except:
#             ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
            rep=Reporting.objects.get(form_data__id_ccr_antt=ficha.strip(),
                                      firm__name=a['Equipe'].strip(),
#                                       occurrence_type__name=a['Classe'].strip(),
                                      company=company).exclude(occurrence_type__occurrence_kind='2')
    
#         rep.form_data['id_ccr_antt'] = ficha
        img=[]
#         if '116' in rep.road_name:
#             ficha_path = 'BR-116/' if novo_caminho else ''
#         else:
#             ficha_path = 'BR-101/' if novo_caminho else ''
        ficha_path = ficha.upper()+'/' if novo_caminho else ''
        for i in range(1,qtd_fotos_por_linhas+1):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': ficha_path+a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i)),
                    'temp':'Foto_'+str(i) # uso exclusivo para monitoração de drenagem profunda separa fotos entre montante e jusante
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,qtd_fotos_por_linhas+1):
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind} ||| {key}")
                                uuid = uuid4()
                                indice=i-1 
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"," | ",rep.form_data[key] if rep.form_data.get(key) else None," | ",a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"))
#                                 print(rep.form_data[key])                            
#                                 print(convert_string(v['inner_api']), [y for x in rep.form_data[key] for y in x.keys()])
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [y for x in rep.form_data[key] for y in x.keys()]:
                                    if len(rep.form_data[key]) == 0:
                                        rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                    else:
                                        rep.form_data[key][indice][convert_string(v['inner_api'])]=[str(uuid)]
                                elif i > len(rep.form_data[key]):
#                                     print('elif')
                                    rep.form_data[key][-1][convert_string(v['inner_api'])] = [] if not rep.form_data[key][-1].get(convert_string(v['inner_api'])) else rep.form_data[key][-1][convert_string(v['inner_api'])]
                                    rep.form_data[key][-1][convert_string(v['inner_api'])].append(str(uuid))
                                else:
                                    rep.form_data[key][indice][convert_string(v['inner_api'])] = [] if not rep.form_data[key][indice].get(convert_string(v['inner_api'])) else rep.form_data[key][indice][convert_string(v['inner_api'])]
                                    rep.form_data[key][indice][convert_string(v['inner_api'])].append(str(uuid))
#                                 print(f"ADICIONADO {rep.form_data[key]}")
                                img.append({
                                    'foto': ficha_path+a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
#                     if len(value) >1 and rep.form_data.get(key):
#                         my_dict={}
# #                         rep.form_data[key] = [{d} for d in rep.form_data.get(key)]
#                         for d in rep.form_data.get(key):
#                             my_dict.update(d)
#                         rep.form_data[key] = [my_dict]

#         print(rep.number,rep.form_data)
        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),ficha,i+2,e)
        erros.append(a)

  0%|          | 0/2099 [00:00<?, ?it/s]

In [16]:
len(erros),len(objects),objects[1],objects[1][0].__dict__

(0,
 2099,
 (<Reporting: [CCR - Rio Sp] CCR-TER-2024.00002 - 2023-12-02 03:00:00+00:00>,
  [{'foto': 'TP 101 RJ 380+800 S1.WMF',
    'data_foto': None,
    'tipo_foto': None,
    'des_foto': None,
    'uuid_foto': UUID('571609f5-5735-479e-a117-88b28572ada8')}]),
 {'_state': <django.db.models.base.ModelState at 0x7911de3c0730>,
  'uuid': UUID('75b4ee82-c8a8-4607-979d-200c82258ef0'),
  'number': 'CCR-TER-2024.00002',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-101 RJ',
  'road_id': 2040,
  'km': 380.8,
  'end_km': 381.6,
  'km_reference': None,
  'project_km': None,
  'project_end_km': None,
  'point': <Point object at 0x7911de345090>,
  'direction': '1',
  'lane': '99',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('709eb517-9d0d-43cd-b8aa-9bc84bcd020a'),
  'firm_id': UUID('d8e33492-4339-40fb-a3e0-312fa2c8d706'),
  'occurrence_type_id': UUID('a324e8df-cbbf-4c91-b294-92d1d656e158'),
  'form_data': {'uf': 'RJ',
   'index':

In [17]:
extensoes=['.JPG','.jpeg','.png','.WMF']
apt=[]
fotos_apt=[]
for (reporting, images) in tqdm(objects):
    can_save=False
    
    for image in images:
        
        foto_file = image.get('foto') if any(image.get('foto').lower().endswith(ext) for ext in ['.JPG','.jpeg','.png','.txt','.jpg','.wmf','.pdf']) else image.get('foto') + '.jpg'
#         print(foto_file,' | ',caminho_imagens+foto_file)
        if ReportingFile.objects.filter(reporting=reporting,
                                        upload__icontains=foto_file.replace('.jpg','').replace('.jpeg','').replace('.png','')).exists() or image.get('foto') == '':
            continue
        reporting_file = ReportingFile(
            uuid= image.get('uuid_foto',uuid4()),
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else '',
            datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            is_shared=False,
            point=reporting.point,
            upload=caminho_imagens+foto_file
        )

        try:
            reporting_file.upload.size
            fotos_apt.append(reporting_file)
#             reporting_file.save()
#             print(reporting_file.upload,reporting_file.uuid)
            can_save=True
        except Exception as e:
            print('Não encontrado1: ',reporting_file.upload)
            for ext in extensoes:
                try:
                    reporting_file.upload = caminho_imagens+foto_file.replace('.jpg',ext)
                    reporting_file.upload.size
                    fotos_apt.append(reporting_file)
#                     reporting_file.save()
                    can_save=True
                    break
                except:
                    print('Não encontrado2: ',reporting_file.upload)
                    continue
    if can_save:
        print('append',reporting.number)
        apt.append(reporting)
#         with DisableSignals():
#             reporting.save()

  0%|          | 0/2099 [00:00<?, ?it/s]

append CCR-TER-2024.00001
append CCR-TER-2024.00002
append CCR-TER-2024.00003
append CCR-TER-2024.00004
append CCR-TER-2024.00005
append CCR-TER-2024.00008
append CCR-TER-2024.00006
append CCR-TER-2024.00007
append CCR-TER-2024.00009
append CCR-TER-2024.00010
append CCR-TER-2024.00011
append CCR-TER-2024.00012
append CCR-TER-2024.00013
append CCR-TER-2024.00014
append CCR-TER-2024.00015
append CCR-TER-2024.00016
append CCR-TER-2024.00017
append CCR-TER-2024.00018
append CCR-TER-2024.00019
append CCR-TER-2024.00020
append CCR-TER-2024.00021
append CCR-TER-2024.00022
append CCR-TER-2024.00023
append CCR-TER-2024.00024
append CCR-TER-2024.00025
append CCR-TER-2024.00026
append CCR-TER-2024.00027
append CCR-TER-2024.00028
append CCR-TER-2024.00029
append CCR-TER-2024.00030
append CCR-TER-2024.00031
append CCR-TER-2024.00032
append CCR-TER-2024.00033
append CCR-TER-2024.00034
append CCR-TER-2024.00035
append CCR-TER-2024.00036
append CCR-TER-2024.00037
append CCR-TER-2024.00038
append CCR-T

append CCR-TER-2024.00318
append CCR-TER-2024.00319
append CCR-TER-2024.00320
append CCR-TER-2024.00321
append CCR-TER-2024.00322
append CCR-TER-2024.00323
append CCR-TER-2024.00324
append CCR-TER-2024.00325
append CCR-TER-2024.00326
append CCR-TER-2024.00327
append CCR-TER-2024.00328
append CCR-TER-2024.00329
append CCR-TER-2024.00330
append CCR-TER-2024.00331
append CCR-TER-2024.00332
append CCR-TER-2024.00333
append CCR-TER-2024.00334
append CCR-TER-2024.00335
append CCR-TER-2024.00336
append CCR-TER-2024.00337
append CCR-TER-2024.00338
append CCR-TER-2024.00339
append CCR-TER-2024.00340
append CCR-TER-2024.00341
append CCR-TER-2024.00342
append CCR-TER-2024.00343
append CCR-TER-2024.00344
append CCR-TER-2024.00345
append CCR-TER-2024.00346
append CCR-TER-2024.00347
append CCR-TER-2024.00348
append CCR-TER-2024.00349
append CCR-TER-2024.00350
append CCR-TER-2024.00351
append CCR-TER-2024.00352
append CCR-TER-2024.00353
append CCR-TER-2024.00354
append CCR-TER-2024.00355
append CCR-T

append CCR-TER-2024.00635
append CCR-TER-2024.00636
append CCR-TER-2024.00637
append CCR-TER-2024.00638
Não encontrado1:  arquivos_monitoracao/2024/ocupacoes_v4/TP 101 RJ 579+420 N1.WMF
Não encontrado2:  arquivos_monitoracao/2024/ocupacoes_v4/TP 101 RJ 579+420 N1.WMF
Não encontrado2:  arquivos_monitoracao/2024/ocupacoes_v4/TP 101 RJ 579+420 N1.WMF
Não encontrado2:  arquivos_monitoracao/2024/ocupacoes_v4/TP 101 RJ 579+420 N1.WMF
Não encontrado2:  arquivos_monitoracao/2024/ocupacoes_v4/TP 101 RJ 579+420 N1.WMF
append CCR-TER-2024.00641
append CCR-TER-2024.00640
append CCR-TER-2024.00642
append CCR-TER-2024.00643
append CCR-TER-2024.00644
append CCR-TER-2024.00645
append CCR-TER-2024.00646
append CCR-TER-2024.00647
append CCR-TER-2024.00648
append CCR-TER-2024.00649
append CCR-TER-2024.00650
append CCR-TER-2024.00651
append CCR-TER-2024.00652
append CCR-TER-2024.00653
append CCR-TER-2024.00654
append CCR-TER-2024.00655
append CCR-TER-2024.00656
append CCR-TER-2024.00657
append CCR-TER-202

append CCR-TER-2024.00924
append CCR-TER-2024.00925
append CCR-TER-2024.00926
append CCR-TER-2024.00927
append CCR-TER-2024.00928
append CCR-TER-2024.00929
append CCR-TER-2024.00930
append CCR-TER-2024.00931
append CCR-TER-2024.00932
append CCR-TER-2024.00933
append CCR-TER-2024.00934
append CCR-TER-2024.00935
append CCR-TER-2024.00936
append CCR-TER-2024.00937
append CCR-TER-2024.00938
append CCR-TER-2024.00939
append CCR-TER-2024.00940
append CCR-TER-2024.00943
append CCR-TER-2024.00941
append CCR-TER-2024.00942
append CCR-TER-2024.00944
append CCR-TER-2024.00945
append CCR-TER-2024.00946
append CCR-TER-2024.00947
append CCR-TER-2024.00948
append CCR-TER-2024.00949
append CCR-TER-2024.00950
append CCR-TER-2024.00951
append CCR-TER-2024.00952
append CCR-TER-2024.00953
append CCR-TER-2024.00954
append CCR-TER-2024.00955
append CCR-TER-2024.00956
append CCR-TER-2024.00957
append CCR-TER-2024.00958
append CCR-TER-2024.00959
append CCR-TER-2024.00960
append CCR-TER-2024.00961
append CCR-T

append CCR-TER-2024.01213
append CCR-TER-2024.01214
append CCR-TER-2024.01217
append CCR-TER-2024.01215
append CCR-TER-2024.01216
append CCR-TER-2024.01218
append CCR-TER-2024.01219
append CCR-TER-2024.01220
append CCR-TER-2024.01221
append CCR-TER-2024.01222
append CCR-TER-2024.01223
append CCR-TER-2024.01224
append CCR-TER-2024.01225
append CCR-TER-2024.01226
append CCR-TER-2024.01227
append CCR-TER-2024.01228
append CCR-TER-2024.01229
append CCR-TER-2024.01230
append CCR-TER-2024.01231
append CCR-TER-2024.01232
append CCR-TER-2024.01233
append CCR-TER-2024.01234
append CCR-TER-2024.01235
append CCR-TER-2024.01236
append CCR-TER-2024.01237
append CCR-TER-2024.01238
append CCR-TER-2024.01239
append CCR-TER-2024.01240
append CCR-TER-2024.01241
append CCR-TER-2024.01242
append CCR-TER-2024.01243
append CCR-TER-2024.01244
append CCR-TER-2024.01245
append CCR-TER-2024.01246
append CCR-TER-2024.01247
append CCR-TER-2024.01248
append CCR-TER-2024.01249
append CCR-TER-2024.01250
append CCR-T

append CCR-TER-2024.01518
append CCR-TER-2024.01519
append CCR-TER-2024.01520
append CCR-TER-2024.01521
append CCR-TER-2024.01522
append CCR-TER-2024.01523
append CCR-TER-2024.01524
append CCR-TER-2024.01525
append CCR-TER-2024.01526
append CCR-TER-2024.01527
append CCR-TER-2024.01528
append CCR-TER-2024.01529
append CCR-TER-2024.01530
append CCR-TER-2024.01531
append CCR-TER-2024.01532
append CCR-TER-2024.01533
append CCR-TER-2024.01534
append CCR-TER-2024.01535
append CCR-TER-2024.01536
append CCR-TER-2024.01537
append CCR-TER-2024.01538
append CCR-TER-2024.01539
append CCR-TER-2024.01540
append CCR-TER-2024.01541
append CCR-TER-2024.01542
append CCR-TER-2024.01543
append CCR-TER-2024.01544
append CCR-TER-2024.01545
append CCR-TER-2024.01546
append CCR-TER-2024.01547
append CCR-TER-2024.01548
append CCR-TER-2024.01549
append CCR-TER-2024.01550
append CCR-TER-2024.01551
append CCR-TER-2024.01552
append CCR-TER-2024.01553
append CCR-TER-2024.01554
append CCR-TER-2024.01555
append CCR-T

append CCR-TER-2024.01838
append CCR-TER-2024.01839
append CCR-TER-2024.01840
append CCR-TER-2024.01841
append CCR-TER-2024.01842
append CCR-TER-2024.01846
append CCR-TER-2024.01843
append CCR-TER-2024.01844
append CCR-TER-2024.01845
append CCR-TER-2024.01847
append CCR-TER-2024.01848
append CCR-TER-2024.01849
append CCR-TER-2024.01850
append CCR-TER-2024.01851
append CCR-TER-2024.01852
append CCR-TER-2024.01853
append CCR-TER-2024.01854
append CCR-TER-2024.01855
append CCR-TER-2024.01856
append CCR-TER-2024.01857
append CCR-TER-2024.01858
append CCR-TER-2024.01859
append CCR-TER-2024.01860
append CCR-TER-2024.01861
append CCR-TER-2024.01862
append CCR-TER-2024.01863
append CCR-TER-2024.01864
append CCR-TER-2024.01865
append CCR-TER-2024.01866
append CCR-TER-2024.01867
append CCR-TER-2024.01871
append CCR-TER-2024.01868
append CCR-TER-2024.01869
append CCR-TER-2024.01870
append CCR-TER-2024.01872
append CCR-TER-2024.01873
append CCR-TER-2024.01874
append CCR-TER-2024.01875
append CCR-T

In [18]:
len(fotos_apt),len(apt),apt[0].form_data,apt[0]

(2092,
 2092,
 {'uf': 'RJ',
  'index': 1,
  'notes': 'DEGRADAÇÃO SUPERFICIAL NO LIMITE DA FAIXA DE DOMÍNIO NO KM 381+180.',
  'x_utm': 7470138.0,
  'y_utm': 643610.0,
  'length': 400.0,
  'distance': 10.0,
  'latitude': -22.870454,
  'zona_utm': '23K',
  'longitude': -43.6001,
  'risk_level': '1',
  'vegetation': '2',
  'id_ccr_antt': 'TP 101 RJ 380+800 N1',
  'inclination': 80.0,
  'rupture_case': False,
  'is_contention': False,
  'water_presence': '1',
  'ocorrencia_tipo': ['4'],
  'probable_causes': '1',
  'tipo_terrapleno': '2',
  'condition_dre_sup': '1',
  'height_terrapleno': 6.0,
  'passivo_ambiental': False,
  'terrain_embossing': '2',
  'condition_dre_prof': '1',
  'mandatory_pictures': [{'panoramic_mandatory': ['fd0e0aab-8f68-40d8-8410-b000c456e3ff',
     '8d4266e4-9184-4ad9-94f5-78f2774fa36e',
     '8cba3ec2-0e1c-48e1-8d0c-21f7006c955f']}],
  'vegetation_density': '1',
  'drenagem_subterranea': '2',
  'drenagem_superficial': '2',
  'kind_drenagem_subterranea': '1',
  'croq

In [19]:
fot = bulk_create_with_history(fotos_apt, ReportingFile, default_user=user)
apontamentos = bulk_update_with_history(apt,Reporting, use_django_bulk=True,batch_size=100,user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [17]:
t=[teste.parent.number for teste,_ in objects if teste not in apt]
len(t),t

(0, [])

In [30]:
objects[1][1][0]['temp']

'Foto_1'

In [32]:
[a['foto'] for a in objects[1][1] if "Foto_1" in a['temp'] or "Foto_2" in a['temp']]

['BU 116 RJ 215+200 N 1_F0_MONTANTE.jpg',
 'BU 116 RJ 215+200 N 1_F1_MONTANTE.jpg']

In [20]:
[a['foto'] for a in objects[1][1] if "Foto_1" or "Foto_2" in a.values()]

(<Reporting: [CCR - Rio Sp] CCR-OAC-2024.43976 - 2024-01-05 03:00:00+00:00>,
 [{'foto': 'BU 116 RJ 215+200 N 1_F0_MONTANTE.jpg',
   'data_foto': None,
   'tipo_foto': None,
   'des_foto': 'Foto Montante panoramica',
   'temp': 'Foto_1'},
  {'foto': 'BU 116 RJ 215+200 N 1_F1_MONTANTE.jpg',
   'data_foto': None,
   'tipo_foto': None,
   'des_foto': 'Foto Montante detalhe',
   'temp': 'Foto_2'},
  {'foto': 'BU 116 RJ 215+200 N 1_F0_JUSANTE.jpg',
   'data_foto': None,
   'tipo_foto': None,
   'des_foto': 'Foto jusante panoramica',
   'temp': 'Foto_3'},
  {'foto': 'BU 116 RJ 215+200 N 1_F1_JUSANTE.jpg',
   'data_foto': None,
   'tipo_foto': None,
   'des_foto': 'Foto jusante detalhe',
   'temp': 'Foto_4'}])

In [18]:
for (reporting, images) in tqdm(objects[:4]):
    print(reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='montante').first())
    print(reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='jusante').first())

  0%|          | 0/4 [00:00<?, ?it/s]

[CCR - Rio Sp] CCR-OAC-2024.42686 - CCR-OAC-2024.42687 (Montante)
[CCR - Rio Sp] CCR-OAC-2024.42686 - CCR-OAC-2024.42688 (Jusante)
[CCR - Rio Sp] CCR-OAC-2024.42689 - CCR-OAC-2024.42690 (Montante)
[CCR - Rio Sp] CCR-OAC-2024.42689 - CCR-OAC-2024.42691 (Jusante)
[CCR - Rio Sp] CCR-OAC-2024.42692 - CCR-OAC-2024.42693 (Montante)
[CCR - Rio Sp] CCR-OAC-2024.42692 - CCR-OAC-2024.42694 (Jusante)
[CCR - Rio Sp] CCR-OAC-2024.42695 - CCR-OAC-2024.42696 (Montante)
[CCR - Rio Sp] CCR-OAC-2024.42695 - CCR-OAC-2024.42697 (Jusante)


# Vinculo da imagem com os ReportingRelation

In [33]:
extensoes=['.JPG','.jpeg','.png']
apt=[]
fotos_apt=[]
for (reporting, images) in tqdm(objects):
    can_save=False
    
    for image in images:
        if "Foto_1" in image['temp'] or "Foto_2" in image['temp']:
            rep_montante=reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='montante').first().child
            foto_file = image.get('foto') if any(image.get('foto').lower().endswith(ext) for ext in ['.JPG','.jpeg','.png','.txt','.jpg','.wmf','.pdf']) else image.get('foto') + '.jpg'
    #         print(foto_file,' | ',caminho_imagens+foto_file)
            if ReportingFile.objects.filter(reporting=rep_montante,
                                            upload__icontains=foto_file.replace('.jpg','').replace('.jpeg','').replace('.png','')).exists() or image.get('foto') == '':
                continue

            reporting_file = ReportingFile(
                uuid= image.get('uuid_foto',uuid4()),
                created_by=user,
                reporting=rep_montante,
                description=image.get('des_foto') if image.get('des_foto') else '',
                datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
                kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
                km=reporting.km,
                is_shared=False,
                point=reporting.point,
                upload=caminho_imagens+foto_file
            )

            try:
                reporting_file.upload.size
                fotos_apt.append(reporting_file)
    #             reporting_file.save()
    #             print(reporting_file.upload,reporting_file.uuid)
                can_save=True
            except Exception as e:
                print('Não encontrado1: ',reporting_file.upload)
                for ext in extensoes:
                    try:
                        reporting_file.upload = caminho_imagens+foto_file.replace('.jpg',ext)
                        reporting_file.upload.size
                        fotos_apt.append(reporting_file)
    #                     reporting_file.save()
                        can_save=True
                        break
                    except:
                        print('Não encontrado2: ',reporting_file.upload)
                        continue
        elif "Foto_3" in image['temp'] or "Foto_4" in image['temp']:
            rep_jusante=reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='jusante').first().child
            foto_file = image.get('foto') if any(image.get('foto').lower().endswith(ext) for ext in ['.JPG','.jpeg','.png','.txt','.jpg','.wmf','.pdf']) else image.get('foto') + '.jpg'
    #         print(foto_file,' | ',caminho_imagens+foto_file)
            if ReportingFile.objects.filter(reporting=rep_jusante,
                                            upload__icontains=foto_file.replace('.jpg','').replace('.jpeg','').replace('.png','')).exists() or image.get('foto') == '':
                continue

            reporting_file = ReportingFile(
                uuid= image.get('uuid_foto',uuid4()),
                created_by=user,
                reporting=rep_jusante,
                description=image.get('des_foto') if image.get('des_foto') else '',
                datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
                kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
                km=reporting.km,
                is_shared=False,
                point=reporting.point,
                upload=caminho_imagens+foto_file
            )

            try:
                reporting_file.upload.size
                fotos_apt.append(reporting_file)
    #             reporting_file.save()
    #             print(reporting_file.upload,reporting_file.uuid)
                can_save=True
            except Exception as e:
                print('Não encontrado1: ',reporting_file.upload)
                for ext in extensoes:
                    try:
                        reporting_file.upload = caminho_imagens+foto_file.replace('.jpg',ext)
                        reporting_file.upload.size
                        fotos_apt.append(reporting_file)
    #                     reporting_file.save()
                        can_save=True
                        break
                    except:
                        print('Não encontrado2: ',reporting_file.upload)
                        continue
        else:
            print('eita',reporting.number)

  0%|          | 0/7544 [00:00<?, ?it/s]

Não encontrado1:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F0_JUSANTE.jpg
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F0_JUSANTE.JPG
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F0_JUSANTE.jpeg
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F0_JUSANTE.png
Não encontrado1:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F1_JUSANTE.jpg
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F1_JUSANTE.JPG
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F1_JUSANTE.jpeg
Não encontrado2:  arquivos_monitoracao/2024/oac/BU 116 SP 184+210 N 1_F1_JUSANTE.png


In [35]:
reporting,fotos_apt[-3].__dict__,fotos_apt[-3].reporting

(<Reporting: [CCR - Rio Sp] CCR-OAC-2024.66602 - 2024-01-22 03:00:00+00:00>,
 {'_state': <django.db.models.base.ModelState at 0x7c74f2bb2730>,
  'uuid': UUID('d5fdf21d-2854-450e-91cb-c8ee4a206472'),
  'reporting_id': UUID('a6ae67b6-7eb9-4874-bee2-827e41234c1a'),
  'description': 'Foto Montante detalhe',
  'md5': '',
  'upload': <FieldFile: arquivos_monitoracao/2024/oac/BU 116 SP 231+616 S 2_F1_MONTANTE.jpg>,
  'uploaded_at': None,
  'datetime': datetime.datetime(2000, 1, 1, 0, 0),
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'include_dnit': True,
  'include_rdo': False,
  'km': 231.616,
  'point': <Point object at 0x7c7504556490>,
  'kind': 'outro',
  'is_shared': False},
 <Reporting: [CCR - Rio Sp] CCR-OAC-2024.66603 - 2024-01-22 03:00:00+00:00>)

In [36]:
fot = bulk_create_with_history(fotos_apt, ReportingFile, default_user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [49]:
teste_mon=reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='montante').first()
teste_jus=reporting.reporting_relation_parent.filter(reporting_relation__name__icontains='jusante').first()

In [52]:
reporting,teste_mon.child,teste_jus.child

(<Reporting: [CCR - Rio Sp] CCR-OAC-2024.43961 - 2024-01-25 03:00:00+00:00>,
 <Reporting: [CCR - Rio Sp] CCR-OAC-2024.43962 - 2024-01-25 03:00:00+00:00>,
 <Reporting: [CCR - Rio Sp] CCR-OAC-2024.43963 - 2024-01-25 03:00:00+00:00>)

In [47]:
reporting,rep_montante.parent,rep_montante.child

AttributeError: 'Reporting' object has no attribute 'child'

In [86]:
reporting.form_data

{'uf': 'RJ',
 'cpf': '2332424',
 'lat': 1,
 'city': 'Itaguaí',
 'long': 2,
 'name': 'jose renato',
 'zone': '1',
 'sheet': 1,
 'state': '1',
 'address': 'rua rj',
 'cartory': '3',
 'celular': '33333333333',
 'therapy': [{'action_date': '2023-11-21T17:22:00.000Z',
   'description': 'abc',
   'action_number': 1,
   'occurrence_type': '66efad60-cf98-4807-9755-5ecbfbb2c02e',
   'treatment_images': ['2d0d73df-b651-4155-af17-e50668526eba']}],
 'matricule': 2,
 'ocupation': '7',
 'situation': '1',
 'telephone': '2222222222',
 'owner_fone': '1111111111',
 'id_ccr_antt': 'OC 101 RJ 400+000 S ',
 'postal_code': '0000000000',
 'qual_fx_dom': 'yyy',
 'city_of_prop': 'rj',
 'observations': '123',
 'titular_kind': '2',
 'lane_occupied': '1',
 'lane_extension': 1,
 'ocupation_kind': '6',
 'characteristics': '3',
 'ocupation_others': 'aa',
 'ocupation_kind_others': 'xx',
 'utilization_condition': '8',
 'lane_extension_not_edified': 2,
 'lane_extension_edified_area': 3,
 'utilization_condition_other': 

In [92]:
reporting=Reporting.objects.get(uuid='221b8853-75b4-4d52-a512-9bf2e62a20b6')
caminho_imagens+'OC116SP_058+622_PN_01_R0 (2).jpg'

'arquivos_monitoracao_homolog/OC116SP_058+622_PN_01_R0 (2).jpg'

In [93]:
reporting_file = ReportingFile(
            uuid= uuid4(),
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else '',
            datetime=image.get('data_foto',datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'),datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            point=reporting.point,
            is_shared=False,
            upload=caminho_imagens+'OC116SP_058+622_PN_01_R0 (2).jpg'
        )

In [94]:
reporting_file.save()

In [95]:
reporting_file.uuid


UUID('18e8ecd0-d9ca-450c-b3b4-3e29839ece87')

In [13]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha[4541:])):
    try:
        ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Código Jusante')
        try:
            parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a.get('Código do Inventário para vínculo com apontamento','')
            rep=Reporting.objects.get(parent__number=parent.strip(),
#                                       km=a['km'],
                                      occurrence_type__name=a['Classe'].strip(),company=company)
        except:
#             print(ficha)
            rep=Reporting.objects.get(occurrence_type__name=a['Classe'].strip(),company=company
                                      ,form_data__code=ficha.strip()
#                                       ,parent__number=parent.strip()
#                 ,form_data__id_ccr_antt=ficha.strip()
                                     )
    
        img=[]

        ficha_path = ficha+'/' if novo_caminho else ''
        for i in range(1,qtd_fotos_por_linhas+1):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': ficha_path+a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i))
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,qtd_fotos_por_linhas+1):
#                             print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}")
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
                                uuid = uuid4()
                                indice=i-1
                                #funcionando com um erro  
#                                 print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"," | ",rep.form_data[key]," | ",a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"))
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [list(teste.keys()) for teste in rep.form_data[key]]:
                                    rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                else:
                                    rep.form_data[key][indice][convert_string(v['inner_api'])] = [] if not rep.form_data[key][indice].get(convert_string(v['inner_api'])) else rep.form_data[key][indice][convert_string(v['inner_api'])]
                                    rep.form_data[key][indice][convert_string(v['inner_api'])].append(str(uuid))

                                img.append({
                                    'foto': ficha_path+a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
                    if len(value) >1 and rep.form_data.get(key):
                        my_dict={}
                        for d in rep.form_data.get(key):
                            my_dict.update(d)
                        rep.form_data[key] = [my_dict]


        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),i+2,e,rep)
        erros.append(a)

  0%|          | 0/2412 [00:00<?, ?it/s]

In [14]:
a['Classe']

'Monitoração Drenagem Profunda Elemento'

In [15]:
# import pandas as pd
# pd.DataFrame(data=erros).to_excel(filename+'_erros.xlsx')

In [30]:
foto_file,reporting_file.upload,image.get('foto')

('.jpg', <FieldFile: arquivos_monitoracao/dreProf/.jpg>, '')

In [18]:
# for reporting, images in tqdm(objects):
#     for image in images:
#         image['foto'] = image.get('foto').replace('OAEC','OAE')

  0%|          | 0/533 [00:00<?, ?it/s]

In [15]:
objects[0],objects[0][0].__dict__

((<Reporting: [CCR - Rio Sp] CCR-OAC-2023.06574 - 2022-10-18 03:00:00+00:00>,
  [{'foto': '1_DA 101 RJ 382+283 N 1_foto1.jpg',
    'data_foto': datetime.datetime(2022, 10, 18, 0, 0),
    'tipo_foto': 'Outro',
    'des_foto': 'Panorâmica'},
   {'foto': '1_DA 101 RJ 382+283 N 1_foto2.jpg',
    'data_foto': datetime.datetime(2022, 10, 18, 0, 0),
    'tipo_foto': 'Outro',
    'des_foto': 'Detalhe'}]),
 {'_state': <django.db.models.base.ModelState at 0x7f9dbf3e2590>,
  'uuid': UUID('b6b4ebab-345f-48c9-a0a0-0f853a0347c1'),
  'number': 'CCR-OAC-2023.06574',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-101 RJ',
  'road_id': 2040,
  'km': 382.371,
  'end_km': 382.371,
  'km_reference': None,
  'project_km': 0.0,
  'project_end_km': 0.0,
  'point': <Point object at 0x7f9dbf436370>,
  'direction': '0',
  'lane': '99',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('a08c9011-

In [29]:
u='arquivos_monitoracao/terra/101/TP 101 RJ 389+900 S1/Foto 22-11-2022 10.33.07'
len(u)

76

In [83]:
rep=Reporting.objects.filter(parent__number='CCR-Inv-2023.61752')
# rep.parent = Reporting.objects.get(uuid='00169b3d-9a09-488b-98c7-762760df4efa')
rep

<QuerySet [<Reporting: [CCR - Rio Sp] CCR-EPS-2023.06483 - 2022-11-16 03:00:00+00:00>, <Reporting: [CCR - Rio Sp] CCR-EPS-2023.07093 - 2022-11-23 03:00:00+00:00>]>

In [82]:
erros[0]

{'km': 233.484,
 'km final': 233.489,
 'Status': 'Identificado',
 'Equipe': 'Civil: SGS',
 'Encontrado em': datetime.datetime(2022, 11, 23, 0, 0),
 'Executado em': datetime.datetime(2022, 11, 23, 0, 0),
 'Sentido': 'Canteiro Central',
 'Classe': 'Monitoração de Defensas Metálicas',
 'Faixa': 'Não se aplica',
 'Rodovia': 'BR-116 RJ',
 'Identificação ANTT': 'DM 116 RJ 233+484 CC 2',
 'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.61752',
 'Latitude': -22.666961,
 'Longitude': -43.842898,
 'X UTM': 573233.31,
 'Y UTM': 7513422.12,
 'Zona UTM': '23K',
 'Comprimento': 4,
 'Altura Média': 0.68,
 'Dispositivo Pontual': None,
 'Tipo de dispositivo pontual': None,
 'Poste Metálico': 'NÃO',
 'Dispositivo Longitudinal': None,
 'Tipo de dispositivo longitudinal': None,
 'Terminal Abatido': 'NÃO',
 'Defensa Defletida': 'NÃO',
 'Absorvedor de Energia': 'NÃO',
 'Aéreo': 'NÃO',
 'Ancoragem em Talude': 'NÃO',
 'Presença De Elemento Refletivo': 'SIM',
 'Tipo de Refletores': 'Monodire

In [45]:
rep.form_data

{'uf': 'UF',
 'index': 1,
 'notes': 'Observações',
 'x_utm': 0.04,
 'y_utm': 0.88,
 'length': 0.16,
 'therapy': [{'treatment_images': ['91882ee6-5802-4df2-8735-241c0e0a704a']}],
 'distance': 0.12,
 'latitude': 0.14,
 'zona_utm': 'Zona UTM',
 'longitude': 0.3,
 'contention': [{'inner_pictures': ['8ccef38a-aa8c-44bd-9ee9-a926b0c407dd']}],
 'risk_level': '4',
 'vegetation': '4',
 'id_ccr_antt': 'Identificação ANTT',
 'inclination': 0.29,
 'dre_kind_sup': '2',
 'rupture_case': False,
 'is_contention': False,
 'water_presence': '6',
 'ocorrencia_tipo': '4',
 'probable_causes': '8',
 'tipo_terrapleno': '1',
 'condition_dre_sup': '3',
 'drainage_pictures': [{'drainage_picture': ['0e188611-8125-4330-a0ed-3d1b73a9f28e']}],
 'height_terrapleno': 0.82,
 'passivo_ambiental': False,
 'terrain_embossing': '2',
 'condition_dre_prof': '3',
 'mandatory_pictures': [],
 'vegetation_density': '1',
 'drenagem_subterranea': '3',
 'drenagem_superficial': '3',
 'occurrence_type_other': 'Tipo de Ocorrência (Ou

In [14]:
# rep[0].form_data['mandatory_pictures']= rep[0].form_data['mandatoryPictures']
# rep[0].save()

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ


In [38]:
apont = rep

In [40]:
apont.form_data['mandatory_pictures']=[]

In [41]:
apont

<Reporting: [CCR - Rio Sp] CCR-TER-2023.02033 - 2023-09-28 14:51:48.205597+00:00>

In [42]:
# apont.save()

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP


In [11]:
reps=ReportingFile.history.filter(reporting__occurrence_type=rep.occurrence_type,upload__icontains='png',history_type='-')
reps.count()

147

In [22]:
repfile=ReportingFile.objects.filter(reporting__occurrence_type=rep.occurrence_type)

In [23]:
for a in erros:
    try:
        a.upload.size
    except:
        a.delete()

In [46]:
for a in tqdm(erros):
        parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
        rep=Reporting.objects.filter(parent__number=parent.strip(),
                          occurrence_type__name=a['Classe'].strip(),company=company).first()
#         rep.delete()

In [76]:
objects=[]
erros=[]
for i,a in enumerate(tqdm(planilha)):
    try:
        try:
            parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
            rep=Reporting.objects.get(parent__number=parent.strip(),
                                      km=a['km'],
                              occurrence_type__name=a['Classe'].strip(),company=company)
        except:
            ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Ficha')
            rep=Reporting.objects.get(form_data__id_ccr_antt=ficha.strip(),
                              occurrence_type__name=a['Classe'].strip(),company=company)
    
        img=[]
        for i in range(1,11):
            # Imagem Geral
            if a.get('Foto_'+str(i)) is not None:

                img.append({
                    'foto': a.get('Foto_'+str(i)),
                    'data_foto': a.get('Data Foto_'+str(i)),
                    'tipo_foto': a.get('Tipo Foto_'+str(i)),
                    'des_foto': a.get('Descrição Foto_'+str(i))
                })
            # Imagem de arrayOfObjects    
            for rets in  ret_campo_foto_array(rep.occurrence_type):
                for key,value in rets.items():
                    key = convert_string(key)
                    temp={}
                    for v in value:
                        for ind in range(1,11):
                            print(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}")
                            if a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}") is not None:
    #                             print(a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{i}"))
                                uuid = uuid4()

                                #funcionando com um erro    
                                if key not in rep.form_data.keys():
                                    rep.form_data[key]=[]
                                if convert_string(v['inner_api']) not in [list(teste.keys())[0] for teste in rep.form_data[key]]:
                                    rep.form_data[key].append({convert_string(v['inner_api']):[str(uuid)]})
                                else:
                                    rep.form_data[key][-1][convert_string(v['inner_api'])].append(str(uuid))

                                img.append({
                                    'foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Foto_{ind}"),
                                    'data_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Data Foto_{ind}"),
                                    'tipo_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Tipo Foto_{ind}"),
                                    'des_foto': a.get(f"{v['main_array']}_{i}: {v['inner_display']} - Descrição Foto_{ind}"),
                                    'uuid_foto': uuid
                                })
                    if len(value) >1 and rep.form_data.get(key):
                        my_dict={}
                        for d in rep.form_data.get(key):
                            my_dict.update(d)
                        rep.form_data[key] = [my_dict]


        objects.append((rep,img))
    except Exception as e:
        print('Erro ao encontrar o apontamento com vinculo:',parent.strip(),i+2,e,rep)
        erros.append(a)

  0%|          | 0/6 [00:00<?, ?it/s]

Anomalias_1: Fotos - Foto_1
Anomalias_1: Fotos - Foto_2
Anomalias_1: Fotos - Foto_3
Anomalias_1: Fotos - Foto_4
Anomalias_1: Fotos - Foto_5
Anomalias_1: Fotos - Foto_6
Anomalias_1: Fotos - Foto_7
Anomalias_1: Fotos - Foto_8
Anomalias_1: Fotos - Foto_9
Anomalias_1: Fotos - Foto_10
Imagens da Monitoração_1: Iluminação Diurna - Foto_1
Imagens da Monitoração_1: Iluminação Diurna - Foto_2
Imagens da Monitoração_1: Iluminação Diurna - Foto_3
Imagens da Monitoração_1: Iluminação Diurna - Foto_4
Imagens da Monitoração_1: Iluminação Diurna - Foto_5
Imagens da Monitoração_1: Iluminação Diurna - Foto_6
Imagens da Monitoração_1: Iluminação Diurna - Foto_7
Imagens da Monitoração_1: Iluminação Diurna - Foto_8
Imagens da Monitoração_1: Iluminação Diurna - Foto_9
Imagens da Monitoração_1: Iluminação Diurna - Foto_10
Imagens da Monitoração_1: Iluminação Noturna - Foto_1
Imagens da Monitoração_1: Iluminação Noturna - Foto_2
Imagens da Monitoração_1: Iluminação Noturna - Foto_3
Imagens da Monitoração_1: 

In [79]:
objects[4],objects[4][0].__dict__

((<Reporting: [CCR - Rio Sp] CCR-DEF-2023.11273 - 2023-11-28 19:28:22+00:00>,
  [{'foto': 'PI 101 RJ 397+280 S 1 RF foto1',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('f905d104-389a-432d-8410-680722491ac1')},
   {'foto': 'PI 101 RJ 397+280 S 1 RF foto2',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('88570876-b040-457d-b948-8a96f05aa66e')},
   {'foto': 'PI 101 RJ 397+280 S 1 RF foto3',
    'data_foto': None,
    'tipo_foto': 'Durante',
    'des_foto': None,
    'uuid_foto': UUID('d0b10d13-fd41-4b5d-8a4b-1382bbe270be')}]),
 {'_state': <django.db.models.base.ModelState at 0x7fcf883556d0>,
  'uuid': UUID('3693020e-16ca-4318-92ab-8b039216977c'),
  'number': 'CCR-DEF-2023.11273',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-101 RJ',
  'road_id': 2040,
  'km': 397.28,
  'end_km': 397.6,
  'km_reference': None,
  'project_km': None,
  'project_end_km': None,

In [55]:
rep.form_data.keys(),key,rep.form_data

(dict_keys(['uf', 'city', 'latitude', 'longitude', 'tipoposte', 'id_ccr_antt', 'modeloposte', 'qtdlampadas', 'tipolampada', 'latitude_end', 'longitude_end', 'qtdluminarias', 'tipoiluminacao', 'atendimentoabnt', 'modeloluminaria', 'potencialampada', 'qtdpostes_monit', 'general_conservation_state', 'fotos_monitoracao']),
 'fotos_monitoracao',
 {'uf': 'Rio de Janeiro',
  'city': 'ITAGUAÍ',
  'latitude': -22.86627016,
  'longitude': -43.7504935,
  'tipoposte': '1',
  'id_ccr_antt': 'PI 101 RJ 397+830 NS 1',
  'modeloposte': 'Poste De Concreto Circular 9x200dan ',
  'qtdlampadas': 22.0,
  'tipolampada': '1',
  'latitude_end': -22.86627016,
  'longitude_end': -43.7504935,
  'qtdluminarias': 22.0,
  'tipoiluminacao': '5',
  'atendimentoabnt': '1',
  'modeloluminaria': '2',
  'potencialampada': '1',
  'qtdpostes_monit': 22,
  'general_conservation_state': '1',
  'fotos_monitoracao': [{'imagem_noturna': ['dbaf4139-ca3c-4bae-9410-4d289e31146e']}]})

In [52]:
key not in rep.form_data.keys()

False

In [75]:
convert_string(v['inner_api'])

'imagem_noturna'

In [61]:
key

'fotos_monitoracao'

In [58]:
rep.form_data[key]

[{'imagem_noturna': ['dbaf4139-ca3c-4bae-9410-4d289e31146e']}]

In [72]:
[list(teste.keys())[0] for teste in rep.form_data[key]]

['imagem_noturna']

In [74]:
convert_string(v['inner_api']) in [list(teste.keys())[0] for teste in rep.form_data[key]]

True

# EXCLUSÃO DE UUID DE IMAGENS QUE NÃO FORAM VINCULADAS AO APONTAMENTO

In [28]:
from helpers.histories import bulk_update_with_history
from copy import deepcopy
from tqdm.notebook import tqdm

In [29]:
def ret_campo_foto_array_v2(classe):
    campos=[]
    inner_campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[convert_string(a['apiName'])] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[convert_string(a['apiName'])].append({"main_array_display_name":a['displayName'],
                                               "inner_api":convert_string(inner['apiName']),
                                               "inner_display":inner['displayName']})
                    inner_campos.append({convert_string(a['apiName']):convert_string(inner['apiName'])})
            campos.append(temp)
    return campos,inner_campos

In [30]:
company=Company.objects.get(name='CCR - Rio Sp')

In [69]:
rep_edit=[]
counter=0
# for a in tqdm(Reporting.objects.filter(company=company,
#                                        occurrence_type__name='Monitoração de Defensas Em OAE')):
for a,_ in tqdm(objects):
    rep_bkp=deepcopy(a)
    #     print(ret_campo_foto_array(a.occurrence_type)[1])
    for b in ret_campo_foto_array_v2(a.occurrence_type)[1]:
        for c,d in b.items():
            # pega a chave do arrayOfObjects se existir
            if a.form_data.get(c):
                # itera sob o arrayOfObjects e o enumera para posterior exclusão
                for i,e in enumerate(a.form_data.get(c)):
                    # se existir a chave da imagem criada no arrayOfObjects e não for 0
                    if e.get(d) is not None and len(e.get(d)) >0:
#                         print(e.get(d))
                        counter+=1
                        # itera sob os uuid de cada imagem e verifica se os reportingFile existe se caso não existe exclui no except
                        for f in e.get(d):
                            try:
                                print(ReportingFile.objects.get(uuid=f).upload.url)
                            except:
                                # remove o uuid que não existe da lista do innerImageArray
                                e.get(d).remove(f)
                                # se estiver vazio a lista do innerImageArray exclui a chave
                                if len(e.get(d))==0:
                                    del a.form_data[c][i][d]
    if a.form_data != rep_bkp.form_data:
        rep_edit.append(a)
print(counter)

  0%|          | 0/11 [00:00<?, ?it/s]

11


In [70]:
len(rep_edit),rep_edit[0].form_data,rep_edit[0].number,d

(11,
 {'uf': 'SP',
  'rail': '4',
  'zona': '1',
  'croqui': [{}],
  'utm_e1': 517499,
  'utm_n1': 7416240,
  'denominacao': 'Ponte sobre Rio Picinguaba',
  'id_ccr_antt': None,
  'nivel_risco': 'BOM',
  'empresa_antt': '2',
  'codigo_agencia': 'OAE 101 SP 008+320 NS 1',
  'drenagem_limpeza': '2',
  'nota_tecnica_laje': '4',
  'mesoestrutura_antt': '5',
  'tipo_inspecao_antt': '3',
  'houve_anteriormente': False,
  'infraestrutura_antt': None,
  'superestrutura_antt': '4',
  'condicoes_conservacao': '2',
  'condicoes_estabilidade': '1',
  'inspecao_especializada': False,
  'monitoracao_atual_antt': '1',
  'oae_numero_codigo_obra': 95,
  'observacoes_adicionais': '-',
  'placa_gabarito_vertical': '1',
  'inspection_year_campaign': 2023,
  'nivel_vibracao_tabuleiro': '1',
  'armadura_exposta_local1_laje': 'Lajes em balanço.',
  'nota_tecnica_nota_estrutural': '4',
  'nota_tecnica_comentarios_gerais': '4',
  'concreto_desagregado_local1_laje': 'Lajes em balanço.',
  'historico_intervencoe

In [71]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=20,user=User.objects.get(username='rlcs'))

In [65]:
from datetime import datetime

def week_and_day_semana(dt):
    dias = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo"]
    isocalendar = dt.isocalendar()
    return isocalendar[0],isocalendar[1],dias[isocalendar[2]]

dt = datetime.now()
# dia = week_and_day_semana(dt)

print(week_and_day_semana(dt))

(2024, 5, 'Quinta-feira')


In [9]:
user=User.objects.get(username="rlcs")
company=Company.objects.get(name='CCR - Rio Sp')

In [10]:
rep_edit=[]
for a in tqdm(planilha[10080:]):
    rep = Reporting.objects.get(number=a['Código Apontamento'],company__name='CCR - Rio Sp')
    if rep.form_data.get('id_ccr_antt'):
        continue
    rep.form_data['id_ccr_antt'] = a['Identificação Poder Regulador']
    rep_edit.append(rep)

  0%|          | 0/11412 [00:00<?, ?it/s]

In [10]:
planilha[0]

{'Código Apontamento': 'CCR-DEF-2024.14440',
 'Identificação Poder Regulador': 'SA 101 RJ 380+310 S 1',
 'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.146740',
 'km': '380,310',
 'km final': '380,310',
 'Status': 'Executado',
 'Equipe': 'Bureau Veritas - Campo',
 'Encontrado em': datetime.datetime(2023, 12, 17, 0, 0),
 'Executado em': datetime.datetime(2023, 12, 17, 0, 0),
 'Sentido': 'Sul',
 'Classe': 'Monitoração de Sinalização Vertical',
 'Faixa': 'Expressa',
 'Rodovia': 'BR-101 RJ',
 'Lado da pista': 'Direito',
 'Lat': -22.8671539204,
 'Long': -43.5922982544,
 'X (UTM)': 644414.42,
 'Y (UTM)': 7470496.65,
 'Zona (UTM)': '23K',
 'Tipo de Monitoração': 'Primeira Monitoração',
 'Ano': 2023,
 'Semestre': 'Segundo Semestre',
 'Local de Implantação': 'Aérea',
 'UF': 'RJ',
 'Data de Fabricação': None,
 'Nome do Fabricante': None,
 'Pertence a concessão': 'SIM',
 'Placa ausente/removida': 'NÃO',
 'Tipo da Placa': 'Indicação',
 'Código da Placa (Indicação)': 'Indicativa

In [15]:
rep_edit[0].form_data

{'uf': 'RJ',
 'lat': -22.8671539204,
 'area': 4.5,
 'long': -43.5922982544,
 'xutm': 644414.42,
 'yutm': 7470496.65,
 'zone': '1',
 'width': 3.0,
 'height': 1.5,
 'average': 89.6,
 'lane_side': 'Direito',
 'sign_type': '1',
 'sign_type2': '1',
 'measure_one': 95.0,
 'measure_two': 99.0,
 'plate_state': '1',
 'second_sign': True,
 'measure_five': 84.0,
 'measure_four': 88.0,
 'plate_format': '2',
 'sign_missing': False,
 'average_third': 84.0,
 'measure_three': 82.0,
 'minimal_value': 46.4,
 'pelicule_kind': '3',
 'support_state': '1',
 'average_second': 694.2,
 'classification': 'Aprovado',
 'pelicule_color': '4',
 'pelicule_state': '2',
 'plate_material': '2',
 'monitoring_kind': '1',
 'support_material': '5',
 'measure_one_third': 89.0,
 'measure_two_third': 74.0,
 'minimal_value_two': 464.0,
 'pelicule_kind_two': '3',
 'belongs_concession': True,
 'implantation_place': '1',
 'measure_five_third': 68.0,
 'measure_four_third': 93.0,
 'measure_one_second': 789.0,
 'measure_two_second':

In [11]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=1000,user=user)

In [27]:
Reporting.objects.get(occurrence_type__name=planilha[0]['Classe'],reporting__company=company)

FieldError: Cannot resolve keyword 'reporting' into field. Choices are: active_inspection, active_inspection_id, active_inspection_of_inventory, address, approval_step, approval_step_id, branch, children, company, company_id, construction, construction_id, created_at, created_by, created_by_id, created_recuperations_with_relation, direction, due_at, due_at_manually_specified, editable, end_km, end_km_manually_specified, excel_imports, excel_reportings, executed_at, firm, firm_id, form_data, form_metadata, found_at, historicalreporting, inspection_jobs, inventory_async_batches, inventory_jobs, job, job_id, keywords, km, km_reference, lane, lot, number, occurrence_type, occurrence_type_id, parent, parent_id, pdf_import, pdf_import_id, point, project_end_km, project_end_km_manually_specified, project_km, reporting_construction_progresses, reporting_files, reporting_messages, reporting_multiple_daily_reports, reporting_quality_assays, reporting_quality_control_exports, reporting_relation_c

In [38]:
apontamentos=set()
for a in tqdm(planilha):
    apontamentos.add(a['Código apontamento '])

  0%|          | 0/198 [00:00<?, ?it/s]

In [76]:
len(apontamentos)

59

In [77]:
ReportingFile.objects.filter(reporting__number__in=apontamentos,reporting__company=company).delete()

(252, {'reportings.ReportingFile': 252})

In [64]:
def ret_campo_foto_array_v2(classe):
    campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[a['apiName']] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[a['apiName']].append({"inner_api":inner['apiName'],
                                               "main_array":a['displayName'],"inner_display":inner['displayName']})
            campos.append(temp)
    return campos

In [65]:
occ=OccurrenceType.objects.get(name='Monitoração de Instalações Elétricas de Edificações')

In [66]:
ret_campo_foto_array_v2(occ)

[{'therapy': [{'inner_api': 'treatmentImages',
    'main_array': 'Anomalias Encontradas',
    'inner_display': 'Fotos'}]}]

In [59]:
rep_edit=[]

for i,a in enumerate(tqdm(planilha)):

#         ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
    ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Código da Agência')
    try:
        parent=a['Código do Inventário para vinculo com apontamento'] if a.get('Código do Inventário para vinculo com apontamento') else a['Código do Inventário para vínculo com apontamento']
        number=a['Código Apontamento'] if a.get('Código Apontamento') else a.get('Código Apontamento ')
        rep=Reporting.objects.get(
#                                         parent__number=parent.strip(),
#                                         form_data__id_ccr_antt=ficha.strip(),
                                    number=number,
#                                       km=a['km'],
    #                               occurrence_type__name=a['Classe'].strip(),
                                    company=company)
    except:
#             ficha=a.get('Identificação ANTT') if a.get('Identificação ANTT') else a.get('Identificação Poder Concedente')
        rep=Reporting.objects.get(form_data__id_ccr_antt=ficha.strip(),
                                  firm__name=a['Equipe'].strip(),
#                                       occurrence_type__name=a['Classe'].strip(),
                                  company=company).exclude(occurrence_type__occurrence_kind='2')

    rep.form_data['id_ccr_antt'] = ficha
    rep_edit.append(rep)

  0%|          | 0/198 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'strip'

In [60]:
rep_edit[0].form_data

{'uf': 'SP',
 'indice': 1,
 'fase_rn': 126.0,
 'fase_rs': 225.0,
 'fase_tn': 127.0,
 'fase_tr': 227.0,
 'fases_n': 126.0,
 'fases_t': 225.0,
 'therapy': [{'outros': True,
   'instalacao': 'Cozinha - Painel',
   'description': 'Sem diagrama de ligação ',
   'outrodetalhe': 'DIAGRAMA DE LIGAÇÃO FALTANDO',
   'treatment_images': ['658da0d4-4fa8-4cd4-876b-134bf82dc272']}],
 'latitude': -23.332559,
 'obs_fase': 'Sistema de Tensão dentro dos parâmetros da ANEEL, \nEntre 95% e 105% da tensão de referência.',
 'obs_spda': 'Sistema de SPDA dentro dos parâmetros da normal NBR 5119',
 'longitude': -46.138317,
 'municipio': 'Jacareí ',
 'aterramento': '2',
 'observacoes': 'Instalações elétricas da edificação esta em BOM estado, consta apenas 1 anomalia',
 'medicao_spda': 2.05,
 'tensao_sistema': '3',
 'sistema_energia': '3',
 'tipo_edificacao': '3',
 'atendenorma_spda': 'Dentro da Norma',
 'atendimento_abnt': '2',
 'atende_norma_fase': '1',
 'atende_norma_neutro': '1',
 'concessionaria_energia': '

In [61]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=30,user=user)

In [2]:
aps = ApprovalStep.objects.get(approval_flow__company__name='CCR - Rio Sp',
                                  approval_flow__target_model__icontains='reporting',name='Compartilhado com ANTT')

In [17]:
Reporting.objects.filter(company__name='CCR - Rio Sp',
                                         occurrence_type__name__icontains='monitoração',
                         approval_step=aps
                                        ).count()

341564